## Fairness Analysis of NamSor's Ethnicity API Endpoint using Aequitas

Part I: Fairness of Ethnicity Endpoint by Ethnicity

In [ ]:
import pandas as pd
import seaborn as sns
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.plotting import Plot

# import warnings; warnings.simplefilter('ignore')

%matplotlib inline

In [ ]:
df = pd.read_csv("data/compas_ethnicity_predictions.csv")
df.head()

In [ ]:
# Non String columns will lead to problems later so we have to find out if there are any
non_attr_cols = ['id', 'model_id', 'entity_id', 'score', 'label_value', 'rank_abs', 'rank_pct']
attr_cols = df.columns[~df.columns.isin(non_attr_cols)]  # index of the columns that are
df.columns[(df.dtypes != object) & (df.dtypes != str) & (df.columns.isin(attr_cols))]

In [ ]:
# And delete them.
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

In [ ]:
# we only want to look at ethnicity here, since that is what we calculated label_value for
df = df.drop(['race_pred', 'first', 'last', 'sex'], axis=1) # if we don't drop the tables, Aequitas thinks these are attributes by which groups should be separated

In [ ]:
df.shape

## Group Metrics

In [ ]:
t = 0.8

In [ ]:
f_pp = df[((df['race'] == 'African-American') & (df['score'] >= t))]
f_pp.count()

In [ ]:
f_pn = df[((df['race'] == 'African-American') & (df['score'] < t))]
f_pn.count()

In [ ]:
f_p = df[((df['race'] == 'African-American') & (df['label_value'] == 1))]
f_p.count()

In [ ]:
f_n = df[((df['race'] == 'African-American') & (df['label_value'] == 0))]
f_n.count()

In [ ]:
f_tn = df[((df['race'] == 'African-American') & (df['score'] < t) & (df['label_value'] == 0) )]
f_tn.count()

In [ ]:
f_tp = df[((df['race'] == 'African-American') & (df['score'] >= t) & (df['label_value'] == 1))]
f_tp.count()

In [ ]:
f_fn = df[((df['race'] == 'African-American') & (df['score'] < t) & (df['label_value'] == 1))]
f_fn.count()

In [ ]:
f_fp = df[((df['race'] == 'African-American') & (df['score'] >= t) & (df['label_value'] == 0))]
f_fp.count()

In [ ]:
g = Group()
xtab, _ = g.get_crosstabs(df, attr_cols=["race"], score_thresholds= {'score': [t]})
absolute_metrics = g.list_absolute_metrics(xtab)

In [ ]:
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

In [ ]:
xtab[['attribute_name', 'attribute_value'] + absolute_metrics].round(2)

### Visualize Group Metrics

In [ ]:
aq_palette = sns.diverging_palette(225, 35, n=2)
aqp = Plot()

In [ ]:
a = aqp.plot_group_metric_all(xtab, ncols=3)

## Disparities of Group Metrics

In [ ]:
b = Bias()
df.head()

In [ ]:
df.dtypes

#### Disparities calculated in relation to a user-specified group for each attribute

In [ ]:
bdf = b.get_disparity_predefined_groups(xtab, original_df=df, 
                                        ref_groups_dict={'race':'Caucasian'}, 
                                        alpha=0.05, check_significance=False, # try with check_significance=True
                                        mask_significance=False)
bdf.style

In [ ]:
# View disparity metrics added to dataframe
bdf[['attribute_name', 'attribute_value'] +
     b.list_disparities(bdf) + b.list_significance(bdf)].style

### Visualizing disparities

In [ ]:
tm_capped = aqp.plot_disparity_all(bdf, attributes=['race'], metrics = 'all', significance_alpha=0.05)

## Fairness Metrics

In [ ]:
f = Fairness()
fdf = f.get_group_value_fairness(bdf)

In [ ]:
parity_detrminations = f.list_parities(fdf)

In [ ]:
fdf[['attribute_name', 'attribute_value'] + absolute_metrics + b.list_disparities(fdf) + parity_detrminations].style

In [ ]:
gaf = f.get_group_attribute_fairness(fdf)
gaf

In [ ]:
gof = f.get_overall_fairness(fdf)
gof

### Visualize fairness

In [ ]:
fg = aqp.plot_fairness_group_all(fdf, ncols=5, metrics = "all")

In [ ]:
a_tm = aqp.plot_fairness_disparity_all(fdf, attributes=['race'], metrics='all', 
                                       significance_alpha=0.05)